In [1]:
import os
import json
import re
import pandas as pd
import numpy as np

from google.cloud import storage
from google.cloud import documentai_v1beta2 as documentai

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\ksaluj1\Documents\work\dev\gv\iacoe_sa.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/ssingh355/code/invoice_processing/iacoe_sa.json"
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199
def parse_ner_json_to_string(parsed_ner):
    ner_string = ""
    flag = True
    for key, value in parsed_ner.items():
        if flag:
            ner_string = ner_string + value + "|" + key
            flag = False
        else:
            ner_string = ner_string + ";" + value + "|" + key
    return ner_string

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    file_name = source_file_name.split('/')[-1]
    destination_blob_name = destination_blob_name + "/" + file_name
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
    gs_bucket_url = "gs://iacoe-cloud-bucket"
    gcs_uri = gs_bucket_url + '/' + destination_blob_name
    return gcs_uri

def extract_data_doc_ai(input_uri, project_id='concrete-tuner-241417'):
         # input_uri='gs://iacoe-cloud-bucket/k'):
    """Process a single document with the Document AI API, including
    text extraction and entity extraction."""

    client = documentai.DocumentUnderstandingServiceClient()
    gcs_source = documentai.types.GcsSource(uri=input_uri)

    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='application/pdf')

    form_extraction_params = documentai.types.FormExtractionParams(
        enabled=True)
    table_extraction_params = documentai.types.TableExtractionParams(
        enabled=True)
    entity_extraction_params = documentai.types.EntityExtractionParams(
        enabled=True)

    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        form_extraction_params=form_extraction_params,
        table_extraction_params=table_extraction_params,
        entity_extraction_params=entity_extraction_params)

    document = client.process_document(request=request)
    data = {}
    data["raw_text"] = (document.text).encode('utf-8')

    def _get_text(el):
        """Convert text offset indexes into text snippets.
        """
        response = ''
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response
    ent_dict = {}
    for entity in document.entities:
        ent_dict[_get_text(entity)] = (entity.mention_text)

    data["entities"] = ent_dict
    form_ent = {}
    for page in document.pages:
        for form_field in page.form_fields:
            key = (_get_text(form_field.field_name), form_field.field_name.confidence)
            value = (_get_text(form_field.field_value), form_field.field_value.confidence)
            form_ent[key] = value
    data["form_data"] = form_ent
    
    out_tables = []
    for page in document.pages:
        for table_num, table in enumerate(page.tables):
            out_table = []
            for row_num, row in enumerate(table.header_rows):
                out_header_row = [_get_text(cell.layout) for cell in row.cells]
                out_table.append(out_header_row)
            for row_num, row in enumerate(table.body_rows):
                out_body_row = [_get_text(cell.layout) for cell in row.cells]
                out_table.append(out_body_row)
            out_tables.append(out_table)

    data["tables"] = out_tables
    
    labels = {}
    for label in document.labels:
        labels[label.name] = label.confidence
    data["labels"] = labels

    return data

In [3]:
# pip install google-cloud-documentai

In [4]:
# pip install google-cloud-storage

In [5]:
# !pip install pandas

In [6]:
# pip install google-cloud

In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
data_files = {}

In [4]:
import time

In [6]:
bucket = "iacoe-cloud-bucket"
destination_blob_name = "iacoe_ocr_ml/invoice_p2p"
file_loc = '/Users/ssingh355/code/invoice_processing/Variation1_concentrix/'
arr_txt =  [x for x in os.listdir(file_loc) if x.endswith(".pdf")]
# print(arr_txt[0])
t1 = time.time()
for i in arr_txt:
    file_path = file_loc + i
    t3 = time.time()
    gcs_uri = upload_blob(bucket, file_path, destination_blob_name)
    data_files[i] = extract_data_doc_ai(gcs_uri)
    print("File {} : time {}".format(i, time.time() - t3))
    

t2 = time.time()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/hsingh151/My_Work/BOL/02_NER_Bol_processing/01_Format_1/01_annotated_samples007_Transaction_2270023542.pdf'

In [11]:
print("total time to process {} files is {}".format(len(arr_txt), t2 - t1))


total time to process 51 files is 336.5832042694092


In [24]:
[*data_files]

['028_CUSA-SINV-028107.pdf',
 '022_CUSA-SINV-027828.pdf',
 '023_CUSA-SINV-027875.pdf',
 '008_CUSA-SINV-027805.pdf',
 '011_CUSA-SINV-027848.pdf',
 '017_CUSA-SINV-027845.pdf',
 '034_CUSA-SINV-027829_Invoice.pdf',
 '012_CUSA-SINV-027850.pdf',
 '033_CUSA-SINV-028137.pdf',
 '040_CUSA-SINV-028135.pdf',
 '014_CUSA-SINV-027865.pdf',
 '037_CUSA-SINV-027832_Invoice (1).pdf',
 '002_CUSA-SINV-027551.pdf',
 '048_CUSA-SINV-028561.pdf',
 '030_CUSA-SINV-028105.pdf',
 '049_CUSA-SINV-028529.pdf',
 '035_CUSA-SINV-028138.pdf',
 '039_CUSA-SINV-028136.pdf',
 '021_CUSA-SINV-027846.pdf',
 '006_CUSA-SINV-027804.pdf',
 '029_CUSA-SINV-028106.pdf',
 '043_CUSA-SINV-028566.pdf',
 '016_CUSA-SINV-027842.pdf',
 '020_CUSA-SINV-027834.pdf',
 '003_CUSA-SINV-027552.pdf',
 '031_CUSA-SINV-028104.pdf',
 '036_CUSA-SINV-027844_Invoice.pdf',
 '047_CUSA-SINV-028528.pdf',
 '046_CUSA-SINV-028562_Annotated.pdf',
 '019_CUSA-SINV-027864.pdf',
 '009_CUSA-SINV-027825.pdf',
 '018_CUSA-SINV-027839.pdf',
 '045_CUSA-SINV-028563.pdf',
 '032

In [14]:
# data_files[[*data_files][3]]['raw_text'].decode().strip().split('\n')[:10]

In [15]:
def doc_ai_ner_parser(raw_text, entities_json,tables):
    
    PO_NUMBER_FLAG = False
    INVOICE_AMOUNT_FLAG = False
    SUBTOTAL_FLAG = False
    TAX_FLAG = False
    
    parsed_data = {}
    get_data = raw_text.strip().split('\n')[:10]
    if len(get_data[0]) < 14:
        parsed_data['Supplier'] = get_data[1]
    else:
        parsed_data['Supplier'] = get_data[0]
    parsed_data['Legal Entity'] = get_data[8][:-5]
    
#     cur = raw_text.find('Invoice\n') + len('Invoice\n')
#     parsed_data['Invoice Curr'] = raw_text[cur:cur+10].split('\n')[0]
    
#     tax_sting = raw_text[raw_text.find('Invoice\n')+len('Invoice\n'):]
#     tax_value = tax_sting[:tax_sting.find('Total')].strip().split('\n')
    
#     parsed_data['Tax Amount'] = tax_value[-3]
#     parsed_data['Tax Rate'] = tax_value[-1]
    
#     print(tax_value)
    parsed_data['Invoice Date'] = get_data[5]#tables[0][0][0].rstrip('\n')
    parsed_data['Invoice Num'] = get_data[6]#tables[0][1][0].rstrip('\n')
#     parsed_data['Invoice Curr'] = tables[1][0][-1].split('(')[1][:3]
#     parsed_data['PO Number'] = tables[1][1][tables[1][0].index('PO Number\n')].rstrip('\n')
#     parsed_data['Invoice Amount'] = tables[1][1][-1].rstrip('\n')

    for key,value in entities_json.items():
#         print(repr(key),'--',repr(value))
        if key.startswith('Invoice Amount'):
            parsed_data['Invoice Curr'] = key.split('(')[1][:3]
            parsed_data['Invoice Amount'] = value
            INVOICE_AMOUNT_FLAG = True
        
        elif key.startswith('PO Number'):
            parsed_data['PO Number'] = value
            PO_NUMBER_FLAG = True 
            
        elif key.startswith('PO Number'):
            parsed_data['PO Number'] = value
            PO_NUMBER_FLAG = True 
            

#     tax_subtotal_string = raw_text.find('Subtotal\nTax\nTotal\n')
    start_index = raw_text.find('Subtotal\nTax\nTotal\n')+ len('Subtotal\nTax\nTotal\n')
    substring1 = raw_text[start_index:]
    subtotal = substring1[:substring1.index('\n')]
    subtotal_index_end = len(subtotal)+ 2
    tax = substring1[subtotal_index_end:substring1.find('\n',subtotal_index_end)]
    
    parsed_data['Tax Amount'] = tax
    parsed_data['Tax Rate'] = int(float(tax))
    
    parsed_data['Line Total/Subtotal'] = subtotal 
    
            
            
    if PO_NUMBER_FLAG == False or INVOICE_AMOUNT_FLAG == False:
        
        for table in tables:
        
            first_line = ''.join(table[0]).rstrip("\n")

            line = first_line.split("\n")
            if PO_NUMBER_FLAG == False:

                if first_line.find('PO Number') != -1:
        #             print(table)
    #                 found_table = True
                    second_line = ''.join(table[1]).rstrip("\n")
                    line1 = second_line.split("\n")
                    if 'PO Number' in line:
                        
        #                 value = line1[line.index('PO Number')]
        #                 print(line1)
        #                 if len(line) == len(line1):
        #                     value = line1[line.index('PO Number')]
        #                 else:
                        value = line1[-3]
                        if value.isdigit():
                            parsed_data['PO Number'] = value
                        else:
                            parsed_data['PO Number'] = value
                    else:
                        for column in line:
                            if column.find("PO Number")!= -1:
        #                         value = line1[line.index(column)].split(" ")[-1]
                                value = line1[-3]
                                if value.isdigit():
                                    parsed_data['PO Number'] = value
#                                     print("PO Number =",value)
                                else:
                                    parsed_data['PO Number'] = value.split(' ')[-1]
#                                     print("PO Number =" ,value.split(' ')[-1])

        #                         print("PO Number = ", value)
                                break 
    
            if INVOICE_AMOUNT_FLAG == False:
                if first_line.find('Invoice Amount') != -1:
                    second_line = ''.join(table[1]).rstrip("\n")
                    line1 = second_line.split("\n")
                    for column in line:
                        if column.find('Invoice Amount') != -1:
                            if len(line1) == len(line):
                                val = line1[line.index(column)]
                            else:
                                val = line1[-1]
                            parsed_data['Invoice Amount'] = val
                            parsed_data['Invoice Curr'] = column.split('(')[1][:3]
            

#               
        
#             parsed_data['Tax Rate'] = '0'
            
#         elif key.startswith('Tax'):
            
#             print("key ", key)
#             print("value ",value)
#             print("****")
#             parsed_data['Tax Amount'] = value
#             parsed_data['Tax Rate'] = '0'
#         elif key.startswith('Subtotal'):
#             parsed_data['Line Total/Subtotal'] = value 
        
            
        
    return parsed_data

In [16]:
[*data_files][0]

'028_CUSA-SINV-028107.pdf'

In [17]:
data_files[[*data_files][3]]['raw_text']

b'CONCENTRIX\nCONCENTRIX CORPORATION US\n44051 Nobel Drive Fremont CA 94538\nDate\nInvoice Number\n2/21/2020\nCUSA-SINV-027805\nBill To:\nUBER TECHNOLOGIES, INC 1455\nMARKET STREET 4TH FLOOR SAN\nFRANCISCO, CA 94103\nRegion\nUS & Canada\nProduct Line\nRides\nDepartment\n60304\nLOB\n1001\nPO Number\n389227\nHours\n12,277\nInvoice Amount (USD)\n$139,975.37\nSubtotal\nTax\nTotal\n$139,975.37\n$0.00\n$139,975.37\nPlease reference invoice number and remit payment to one of the following:\nCheck Payments\nACH Payments: Wire Transfers:\nAttn: Concentrix Corporation Attn:\n32680 Collection Center Drive\nChicago, IL 60693-0326\nACH Payments:\nBank of America\nAttn: Concentrix Corporation\n2000 Clayton Road\nConcord, CA 94520\nABA Routing Number: 121000358\nDepositor Account Number:\n1291539682\nWire Transfers:\nBank of America\nAttn: Concentrix Corporation\n100 West 33rd Street\nNew York, NY 10001\nABA Routing Number: 026009593\nDepositor Account Number:\n1291539682\nPage 1\nFrom:\nDate:\nTo:\n

In [23]:
data_files[[*data_files][3]]['entities']

{'Invoice Number\nDate': 'Date',
 '2/21/2020\nCUSA-SINV-027805': 'CUSA-SINV-027805',
 'Bill To:\nUBER TECHNOLOGIES, INC 1455\nMARKET STREET 4TH FLOOR SAN\nFRANCISCO, CA 94103': 'UBER TECHNOLOGIES, INC 1455\nMARKET STREET 4TH FLOOR SAN\nFRANCISCO, CA 94103',
 'Product Line\nRides': 'Rides',
 'Department\n60304': '60304',
 'Hours\n12,277': '12,277',
 'Invoice Amount (USD)\n$139,975.37': '$139,975.37',
 'Tax\n$0.00': '$0.00',
 'Total\n$139,975.37': '$139,975.37',
 '32680 Collection Center Drive': 'Center Drive',
 'Chicago, IL 60693-0326': '60693-0326',
 'ACH Payments:\nWire Transfers:': 'Wire Transfers:',
 'Bank of America\nBank of America': 'Bank of America',
 'Attn: Concentrix Corporation\nAttn: Concentrix Corporation': 'Corporation\nAttn: Concentrix Corporation',
 '2000 Clayton Road\n100 West 33rd Street': '100 West 33rd Street',
 'Concord, CA 94520': '94520',
 'New York, Concord, CA 94520\nNew York, NY 10001': 'Concord, CA 94520\nNew York, NY 10001',
 'ABA ABA Routing Number: 12100035

In [247]:
# data_files[[*data_files][0]]['form_data']

In [249]:
# data_files[[*data_files][0]]['tables'][0][0][0]

In [250]:
# data_files[[*data_files][7]]['tables']

In [218]:
first_lines = []
count = 0
found_table = False
# found_Invoice = False
for i in range(len([*data_files])):
    tables = data_files[[*data_files][i]]['tables']
    print(i ," ", [*data_files][i])
    for table in tables:
        
        first_line = ''.join(table[0]).rstrip("\n")
        
        line = first_line.split("\n")
        
        if first_line.find('PO Number') != -1:
#             print(table)
            found_table = True
            second_line = ''.join(table[1]).rstrip("\n")
            line1 = second_line.split("\n")
            if 'PO Number' in line:
                
#                 value = line1[line.index('PO Number')]
#                 print(line1)
#                 if len(line) == len(line1):
#                     value = line1[line.index('PO Number')]
#                 else:
                value = line1[-3]
                if value.isdigit():
                    print("PO Number =",value)
                else:
                    print("PO Number =" ,value.split(' ')[-1])
                
#                 print("PO Number = ", value)
            else:
                for column in line:
                    if column.find("PO Number")!= -1:
#                         value = line1[line.index(column)].split(" ")[-1]
                        value = line1[-3]
                        if value.isdigit():
                            print("PO Number =",value)
                        else:
                            print("PO Number =" ,value.split(' ')[-1])

#                         print("PO Number = ", value)
                        break 
            
#         if first_line.find('Invoice Amount') != -1:
#             for column in line:
#                 if column.find('Invoice Amount') != -1:
#                     if len(line1) == len(line):
#                         val = line1[line.index(column)]
#                     else:
#                         val = line1[-1]
#                     print("Invoice Amount", val)
#                     print("Invoice curr", column.split('(')[1][:3])
            
#         if found_table == True:
#             print(count)
#             count+=1
#             break

0   022_CUSA-SINV-027828.pdf
PO Number = 389227
1   023_CUSA-SINV-027875.pdf
PO Number = 357975
2   008_CUSA-SINV-027805.pdf
PO Number = 389227
3   011_CUSA-SINV-027848.pdf
PO Number = 381750
4   028_CUSA-SINV-028107.pdf
PO Number = 389227
5   017_CUSA-SINV-027845.pdf
PO Number = 389227
6   034_CUSA-SINV-027829_Invoice.pdf
PO Number = 381689
7   012_CUSA-SINV-027850.pdf
PO Number = 389227
8   033_CUSA-SINV-028137.pdf
PO Number = 381665
9   040_CUSA-SINV-028135.pdf
PO Number = 381743
10   014_CUSA-SINV-027865.pdf
PO Number = 381750
11   037_CUSA-SINV-027832_Invoice (1).pdf
PO Number = 381689
12   002_CUSA-SINV-027551.pdf
PO Number = 357938
13   048_CUSA-SINV-028561.pdf
PO Number = 381665
14   030_CUSA-SINV-028105.pdf
PO Number = 381665
15   049_CUSA-SINV-028529.pdf
PO Number = 389227
16   035_CUSA-SINV-028138.pdf
PO Number = 389227
17   039_CUSA-SINV-028136.pdf
PO Number = 381689
18   021_CUSA-SINV-027846.pdf
PO Number = 381665
19   006_CUSA-SINV-027804.pdf
PO Number = 381750
20   029_C

In [145]:
data_files[[*data_files][0]]['labels']

{}

In [146]:
[*data_files[[*data_files][0]]]

['raw_text', 'entities', 'form_data', 'tables', 'labels']

In [147]:
[*data_files]

['022_CUSA-SINV-027828.pdf',
 '023_CUSA-SINV-027875.pdf',
 '008_CUSA-SINV-027805.pdf',
 '011_CUSA-SINV-027848.pdf']

In [253]:
# q = '005_960009273.pdf'
# q = '001_960008611_V2_Annotated.pdf'
data = data_files[[*data_files][32]]
a = doc_ai_ner_parser(data['raw_text'].decode(), data['entities'],data['tables'])

a

{'Supplier': 'CONCENTRIX CORPORATION US',
 'Legal Entity': 'UBER TECHNOLOGIES, INC',
 'Invoice Date': '4/30/2020',
 'Invoice Num': 'CUSA-SINV-028563',
 'Tax Amount': '0.00',
 'Tax Rate': 0,
 'Line Total/Subtotal': '$47,788.25',
 'Invoice Curr': 'USD',
 'Invoice Amount': '$47,788.25',
 'PO Number': '402747'}

In [258]:
def table_parser(raw_text, entities_json,tables):
#     get_data = raw_text[raw_text.find('Invoice\n')+len('Invoice\n'):raw_text.find('VAT\n')].strip().split('\n')[1:-1]
    start_index = raw_text.find('Subtotal\nTax\nTotal\n')+ len('Subtotal\nTax\nTotal\n')
    substring1 = raw_text[start_index:]
    subtotal = substring1[:substring1.index('\n')]
#     subtotal_index_end = len(subtotal)+ 2
#     tax = substring1[subtotal_index_end:substring1.find('\n',subtotal_index_end)]
    
    
    table_data = {
        'Description':[],
        'Qty Invoiced':[],
        'Unit Price':[],
        'Line Total/Amount':[],
        
    } 
    
    table_data['Description'].append(" ")
    table_data['Qty Invoiced'].append(subtotal)
    table_data['Unit Price'].append(1)
    table_data['Line Total/Amount'].append(subtotal)
#     for enu,i in enumerate(get_data):
#         if enu % 2 :
#             table_data['Line Total/Amount'].append(i)
#         else:
#             table_data['Description'].append(i)
            
    return pd.DataFrame(table_data).to_dict(orient='records')
    
#     print(table_data)

In [268]:
b = table_parser(data['raw_text'].decode(), data['entities'],data['tables'])
b

[{'Description': ' ',
  'Qty Invoiced': '$47,046.17',
  'Unit Price': 1,
  'Line Total/Amount': '$47,046.17'}]

In [264]:
extract_data = []
extract_table = []

In [265]:
for file,data in data_files.items():
#     print(file)
    a = doc_ai_ner_parser(data['raw_text'].decode(), data['entities'],data['tables'])
#     print(file)
    a['files'] = file
    extract_data.append(a)
    b = table_parser(data['raw_text'].decode(), data['entities'],data['tables'])
#     b['file'] = file
    extract_table.append(b)

In [266]:
pd.DataFrame(extract_data)

,Supplier,Legal Entity,Invoice Date,Invoice Num,Tax Amount,Tax Rate,Line Total/Subtotal,PO Number,Invoice Curr,Invoice Amount,files
0,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",2/26/2020,CUSA-SINV-027828,0.00,0,"$5,192.48",389227,USD,"$5,192.48",022_CUSA-SINV-027828.pdf
1,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",2/26/2020,CUSA-SINV-027875,0.00,0,"$56,494.40",357975,USD,"$56,494.40",023_CUSA-SINV-027875.pdf
2,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",2/21/2020,CUSA-SINV-027805,0.00,0,"$139,975.37",389227,USD,"$139,975.37",008_CUSA-SINV-027805.pdf
3,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",2/26/2020,CUSA-SINV-027848,0.00,0,"$54,352.42",381750,USD,"$54,352.42",011_CUSA-SINV-027848.pdf
4,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",3/17/2020,CUSA-SINV-028107,0.00,0,"$138,373.34",389227,USD,"$138,373.34",028_CUSA-SINV-028107.pdf
5,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",2/26/2020,CUSA-SINV-027845,0.00,0,"$330,976.14",389227,USD,"$330,976.14",017_CUSA-SINV-027845.pdf
6,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",2/26/2020,CUSA-SINV-027829,0.00,0,"$7,795.87",381689,USD,"$7,795.87",034_CUSA-SINV-027829_Invoice.pdf
7,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",2/26/2020,CUSA-SINV-027850,0.00,0,"$779,441.62",389227,USD,"$779,441.62",012_CUSA-SINV-027850.pdf
8,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",3/19/2020,CUSA-SINV-028137,0.00,0,"$98,304.09",381665,USD,"$98,304.09",033_CUSA-SINV-028137.pdf
9,CONCENTRIX CORPORATION US,"UBER TECHNOLOGIES, INC",3/19/2020,CUSA-SINV-028135,0.00,0,"$53,894.29",381743,USD,"$53,894.29",040_CUSA-SINV-028135.pdf


In [269]:
pd.DataFrame(extract_table)

,0
0,"{'Description': ' ', 'Qty Invoiced': '$5,192.48', 'Unit Price': 1, 'Line Total/Amount': '$5,192.48'}"
1,"{'Description': ' ', 'Qty Invoiced': '$56,494.40', 'Unit Price': 1, 'Line Total/Amount': '$56,494.40'}"
2,"{'Description': ' ', 'Qty Invoiced': '$139,975.37', 'Unit Price': 1, 'Line Total/Amount': '$139,975.37'}"
3,"{'Description': ' ', 'Qty Invoiced': '$54,352.42', 'Unit Price': 1, 'Line Total/Amount': '$54,352.42'}"
4,"{'Description': ' ', 'Qty Invoiced': '$138,373.34', 'Unit Price': 1, 'Line Total/Amount': '$138,373.34'}"
5,"{'Description': ' ', 'Qty Invoiced': '$330,976.14', 'Unit Price': 1, 'Line Total/Amount': '$330,976.14'}"
6,"{'Description': ' ', 'Qty Invoiced': '$7,795.87', 'Unit Price': 1, 'Line Total/Amount': '$7,795.87'}"
7,"{'Description': ' ', 'Qty Invoiced': '$779,441.62', 'Unit Price': 1, 'Line Total/Amount': '$779,441.62'}"
8,"{'Description': ' ', 'Qty Invoiced': '$98,304.09', 'Unit Price': 1, 'Line Total/Amount': '$98,304.09'}"
9,"{'Description': ' ', 'Qty Invoiced': '$53,894.29', 'Unit Price': 1, 'Line Total/Amount': '$53,894.29'}"


In [121]:
pd.DataFrame(extract_table)

,0
0,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '588.74'}"
1,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '180.40'}"
2,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '297.39'}"
3,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '736.92'}"
4,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '4,749.37'}"
5,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '193.86'}"
6,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '100.00'}"
7,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '120.64'}"
8,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '1,563.48'}"
9,"{'Description': 'For International VAT Refund Services Delivered', 'Line Total/Amount': '104.05'}"
